In [53]:
#Importing required dependencies

import tweepy
import pymongo
import json
import time

print("Dependencies imported")

Dependencies imported


In [54]:
#TO DO: Add to config file later

consumer_key = "mQXkO4dzpD3OPD1nOIT3fuD0j"
consumer_secret = "obt5r2qT1o878i5Eja7GYropJfCeUX76IXcjXRuHoJjIZnaard"
users_database_name = "users_test"
user_followers_database = "user_followers_test"
tweets_database = "user_tweets_test"

seed_usernames = ["P9Ashwini", 
                  "drashwathcn", 
                  "Tej_AnanthKumar", 
                  "vijeta_at", 
                  "nimmasuresh", 
                  "ShashikalaJolle"]

In [55]:
#Storing in MongoDB database

client = pymongo.MongoClient("mongodb://localhost:27017/")
mydb = client["mydatabase_test"]
user_col = mydb[users_database_name]
user_follower_col = mydb[user_followers_database]
tweets_col = mydb[tweets_database]

In [56]:
#Checking for collections in DB

collist = mydb.list_collection_names()
print(collist)
if users_database_name in collist:
    print("Users collection exists")

else:
    print("ERR: Users collection doesn't exist yet")

['user_tweets_test', 'users_test']
Users collection exists


In [57]:
#OAuth2 Authentication

def get_authenticated_twitter_api():
    auth = tweepy.AppAuthHandler(consumer_key, consumer_secret)
    api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)
    return api

In [58]:
#Save the users info and recent tweets in local DB

def save_user(user_dict):
    try:
        user_info = {"_id":user_dict["id_str"],
                     "name":user_dict["name"], 
                     "screen_name":user_dict["screen_name"], 
                     "followers":user_dict["followers_count"], 
                     "friends":user_dict["friends_count"]}

        x = user_col.insert_one(user_info)

        #Fetching tweets of the user
        api = get_authenticated_twitter_api()
        tweets = api.user_timeline(user_dict["id"])
        for tweet_obj in tweets:
            tweet = tweet_obj._json
            tweet_info = {"_id":tweet["id_str"],
                          "user_id":user_dict["id_str"],
                          "follower_count":user_dict["followers_count"],
                          "retweet_count":tweet["retweet_count"]}

            y = tweets_col.insert_one(tweet_info)
            
        print("User info saved for", user_dict["id"])
            
    except Exception as ex:
        print(ex)

In [66]:
#Performing BFS on twitter users

def scrape_users(user_list):
    count = 0
    api = get_authenticated_twitter_api()
    try:
        while len(user_list)!=0:
            try:
                time.sleep(10)
                if count%100==0:
                    if(count!=0):
                        print("Pausing...")


                while(1):
                    user_id = user_list.pop(0)

                    #Checking if user node not already visited
                    query = {"_id": user_id }
                    query_res = user_col.find(query)

                    qcount = 0
                    for query in query_res:
                        qcount = 1

                    if (qcount)==0:
                        break

                followers = []
                for follower in tweepy.Cursor(api.followers, id=user_id).items():
                    if "bangalore" in follower._json["location"].lower() or "bengaluru" in follower._json["location"].lower():
                        user_list.append(follower._json["id_str"])
                        followers.append(follower._json["id_str"])
                        save_user(follower._json)
                        count = count+1

                #Saving followers of a user        
                follower_info = {"_id":user_id, 
                                 "followers": followers }

                try:
                    user_follower_col.insert_one(follower_info)

                except:
                    print("User's followers already stored")

                if count>5000:
                    break
                    
            except Exception as ex:
                print(ex, " occured inside loop")
                time.sleep(200)
                
    except Exception as ex:
        print(ex, " occured outside loop")

In [67]:
#Fetching user ids of popular seed users

def initialise(seed_usernames):
    seed_users = []
    api = get_authenticated_twitter_api()
    for seed in seed_usernames:
        res = api.get_user(seed)
        #print(res)
        seed_users.append(res._json["id_str"])
        
    print(seed_users)
    return seed_users

In [48]:
#Fetching seed ids

seed_users = initialise(seed_usernames)

['1213502011', '2885971046', '1067115424339435520', '2307766760', '43705720', '704994405065760768']


In [68]:
#Drop the existing collections

user_col.drop()
user_follower_col.drop()
tweets_col.drop()

In [ ]:
seed_users = ['1213502011', '2885971046', '1067115424339435520', '2307766760', '43705720', '704994405065760768']

scrape_users(seed_users)

User info saved for 66338610
User info saved for 849157018833342464
User info saved for 3275260856
User info saved for 364720514
User info saved for 1218462519104159745
User info saved for 139092034
User info saved for 624649724
User info saved for 51379431
User info saved for 1216368239527133186
User info saved for 393845265
User info saved for 1217712610461638656
User info saved for 1124704857552592896
User info saved for 146913591
User info saved for 1041641552760647681
User info saved for 604375461
User info saved for 2546737348
Not authorized.
User info saved for 1208802953181585408


Rate limit reached. Sleeping for: 750


Failed to send request: ('Connection aborted.', OSError("(104, 'ECONNRESET')"))  occured inside loop
User info saved for 1219183196035678208
User info saved for 102461702
User info saved for 982651114884493318
User info saved for 849689042
User info saved for 1017804911889661952
User info saved for 735465065177055234
User info saved for 962380510264725504
User info saved for 987409568907980800
User info saved for 4801821314
User info saved for 3406042757
User info saved for 108263125
User info saved for 1027391207851347969
User info saved for 87880072
User info saved for 1105413755251367939
User info saved for 1000611295228444677
User info saved for 1082120503316307974
User info saved for 874276461393215489
User info saved for 1926118028
User info saved for 1085158061671514112
User info saved for 385916326
User info saved for 2507122206
User info saved for 1107270334821076992
User info saved for 884853999379152896
User info saved for 916119808344694784
User info saved for 1124162937327

Rate limit reached. Sleeping for: 249


User info saved for 460987051
User info saved for 1515863820
User info saved for 1076091912795443205
User info saved for 105810584
User info saved for 140844387
Not authorized.
User info saved for 943030634
User info saved for 83779942
Not authorized.
User info saved for 998758638901542912
User info saved for 1231809893230759936
User info saved for 1209866474967289856
User info saved for 784009104649957377
User info saved for 190585672
User info saved for 1143754776103936000
User info saved for 1065269578215739393
User info saved for 777434340665864193
User info saved for 1098731642464591872
User info saved for 975961359279013888
User info saved for 1140527786316091392
User info saved for 1107488091600650240
User info saved for 2530253768
User info saved for 1059112902697709574
User info saved for 1096348585350725632
User info saved for 308348946
User info saved for 256851100
User info saved for 1211142566533459971
User info saved for 815601558184300544
User info saved for 110948243965

Rate limit reached. Sleeping for: 360


In [ ]:
for x in tweets_col.find():
    print(x) 